## Задание по теме «Коллаборативная фильтрация»

ПАКЕТ SURPRISE

- используйте данные MovieLens 1M
- можно использовать любые модели из пакета
- получите RMSE на тестовом сете 0.87 и ниже

Комментарий преподавателя :
В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.

In [1]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, names=['movieId', 'title', 'genres'])
movies.set_index('movieId')
movies.head()

C:\Users\Polina.DESKTOP-UGBOPLJ\anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  3883 non-null   int64 
 1   title    3883 non-null   object
 2   genres   3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [4]:
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, names=['userId','movieId','rating','timestamp'])
ratings.head()

C:\Users\Polina.DESKTOP-UGBOPLJ\anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   userId     1000209 non-null  int64
 1   movieId    1000209 non-null  int64
 2   rating     1000209 non-null  int64
 3   timestamp  1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


In [6]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [7]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [8]:
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),5.0
1,6.0,Toy Story (1995),4.0
2,8.0,Toy Story (1995),4.0
3,9.0,Toy Story (1995),5.0
4,10.0,Toy Story (1995),5.0


In [9]:
print(ratings.rating.min(), ratings.rating.max())

1 5


Вначале воспроизведу код с лекции. Это будет тот вариант, от результатов которого буду отталкиваться

In [10]:
reader = Reader(rating_scale=(1.0, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [11]:
trainset, testset = train_test_split(data, test_size=.2)

In [12]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [13]:
test_pred = algo.test(testset)

In [14]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8904


0.8903981440946417

In [15]:
assert accuracy.rmse(test_pred, verbose=True)<0.87, 'работаем дальше'

RMSE: 0.8904


AssertionError: работаем дальше

Оценим качество с помощью кросс-валидации

In [16]:
from surprise.model_selection import cross_validate

In [17]:
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8898  0.8873  0.8907  0.8889  0.8880  0.8889  0.0012  
Fit time          162.57  163.03  166.30  170.15  164.52  165.32  2.75    
Test time         164.85  169.36  170.73  166.18  166.81  167.58  2.15    


{'test_rmse': array([0.88977517, 0.88731303, 0.89068745, 0.88894812, 0.8879751 ]),
 'fit_time': (162.56750392913818,
  163.03446006774902,
  166.30397295951843,
  170.1497664451599,
  164.5220696926117),
 'test_time': (164.84638690948486,
  169.35741591453552,
  170.72738003730774,
  166.17723488807678,
  166.80795693397522)}

Все-таки на датасете 1М модель обучается долго. И на сравнение результатов уйдет много времени. Попробую уменьшить датасет хотя бы на порядок, и если удастся подобрать модель, которая значимо скинет rmse, можно будет применить уже её к полному датасету.

In [18]:
df_small = dataset.sample(n=100000)

In [19]:
df_small.shape

(100000, 3)

In [20]:
data_sm = Dataset.load_from_df(df_small, reader)

In [21]:
trainset_2, testset_2 = train_test_split(data_sm, test_size=.2)

In [22]:
algo_2 = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo_2.fit(trainset_2)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [23]:
accuracy.rmse(algo_2.test(testset_2), verbose=True)

RMSE: 1.0943


1.0942959612385121

Пробую разные параметры пока для одного алгоритма KNNWithMeans. Хорошо, что в surprise так же присутствует такая опция как GridSearchCV :) 

In [24]:
from surprise.model_selection import GridSearchCV

In [25]:
sim_options = {'name':['cosine', 'msd', 'pearson']}

In [26]:
param_grid = {'k':[10, 30, 50, 70, 100], 'sim_options': sim_options}

In [27]:
gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], cv=5)

In [28]:
gs.fit(data_sm)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson si

In [29]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.0095163790865593
{'k': 100, 'sim_options': {'name': 'cosine', 'user_based': True}}


Было RMSE 1.085, стало 1.009. Запомним эту комбинацию параметров дл KNNWithMeans и попробуем что-нибудь еще.

Посмотрим на другие предиктивные алгоритмы, которые есть в пакете 

In [30]:
from surprise import KNNBasic, SVD, NMF

In [31]:
algo_dict = {'KNN basic':KNNBasic(), 'SVD':SVD(), 'NMF': NMF()}
for name_i, algo_i in algo_dict.items():
    algo_i.fit(trainset_2)
    print(f'{name_i}: RMSE = {accuracy.rmse(algo_i.test(testset_2), verbose=True)}')
    

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0356
KNN basic: RMSE = 1.0355880359684357
RMSE: 0.9516
SVD: RMSE = 0.9515744689734791
RMSE: 1.0393
NMF: RMSE = 1.0392912729108226


С параметрами по умолчанию лучший результат у SVD. Посмотрим, получится ли улучшить результат, подбирая параметры с помощью GridSearchCV

In [32]:
param_grid_svd = {'n_factors': [80, 100, 120],
                  'n_epochs': [5, 10, 20], 
                  'lr_all': [0.002, 0.005, 0.01],
                  'reg_all': [0.02, 0.05, 0.1]
                 }

In [33]:
gs_svd = GridSearchCV(SVD, param_grid_svd, measures=['RMSE'], cv=5)

In [34]:
gs_svd.fit(data_sm)

In [35]:
print(gs_svd.best_score['rmse'])
print(gs_svd.best_params['rmse'])

0.9466668326988902
{'n_factors': 80, 'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.1}


По сравнению с параметрами по умолчанию результат выполнения SVD (RMSE 0.947) улучшен не так сильно, как этого хотелось бы. Но он заметно лучше, чем RMSE 1.0943 для первоначального KNN. Попробую переобучить алгоритм с найденными параметрами на исходном датасете 1М

In [36]:
svd_best_params = SVD(n_factors=80, n_epochs=20, lr_all=0.01, reg_all=0.1)

In [37]:
cross_validate(svd_best_params, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8798  0.8803  0.8808  0.8809  0.8820  0.8808  0.0007  
Fit time          65.51   64.75   65.62   65.71   66.57   65.63   0.58    
Test time         3.60    3.10    3.38    3.61    3.66    3.47    0.21    


{'test_rmse': array([0.8797652 , 0.8803325 , 0.88081068, 0.88094381, 0.8819693 ]),
 'fit_time': (65.50628304481506,
  64.74503326416016,
  65.61884641647339,
  65.71075487136841,
  66.57282567024231),
 'test_time': (3.601288318634033,
  3.1020448207855225,
  3.3823134899139404,
  3.605949640274048,
  3.6612184047698975)}

Результаты лучше, чем были, но все-таки они не опустились ниже уровня 0,87, что жаль. У нас есть в запасе параметры, найденные GridSearch, для алгоритма KNN, попробую их. 

In [38]:
knn_best_params = KNNWithMeans(k=100, sim_options= {'name': 'cosine', 'user_based': True})

In [39]:
cross_validate(knn_best_params, data, measures=['RMSE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9302  0.9293  0.9288  0.9272  0.9314  0.9294  0.0014  
Fit time          155.94  156.22  156.95  152.12  158.32  155.91  2.07    
Test time         205.16  202.79  204.41  201.61  202.81  203.36  1.27    


{'test_rmse': array([0.930169  , 0.92932093, 0.92879388, 0.92720116, 0.93140076]),
 'fit_time': (155.9358081817627,
  156.21815180778503,
  156.94791078567505,
  152.11530542373657,
  158.3233244419098),
 'test_time': (205.1585259437561,
  202.79371976852417,
  204.41269493103027,
  201.60999178886414,
  202.81285881996155)}

Эта попытка тоже не привела к заданному результату. Предположительно есть смысл вернуться к алгоритму SVD, который для уменьшенного датасета давал лучший результат, и попробовать покрутить его параметры.

In [40]:
for name_i, algo_i in algo_dict.items():
    algo_i.fit(trainset)
    print(f'{name_i}: RMSE = {accuracy.rmse(algo_i.test(testset), verbose=True)}')

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9228
KNN basic: RMSE = 0.9227969386269387
RMSE: 0.8745
SVD: RMSE = 0.8745041223051825
RMSE: 0.9148
NMF: RMSE = 0.9148308662053665


С параметрами по умолчанию SVD и на большом датасете выигрывает у других алгоритмов.

Чтобы поиск параметров был не совсем рандомный, возьму для сетки параметров значения близкие к тем, которые давали наилучший результат после GridSearchCV для уменьшенного датасета

In [41]:
param_grid_svd_new = {'n_factors': [50, 60, 70, 80],
                  'n_epochs': [20, 30, 40], 
                  'lr_all': [0.01, 0.02, 0.03],
                  'reg_all': [0.1, 0.2]
                 }

In [42]:
gs_svd_new = GridSearchCV(SVD, param_grid_svd_new, measures=['RMSE'], cv=5)

In [43]:
gs_svd_new.fit(data)

In [44]:
print(gs_svd_new.best_score['rmse'])
print(gs_svd_new.best_params['rmse'])

0.8691878549499282
{'n_factors': 80, 'n_epochs': 40, 'lr_all': 0.01, 'reg_all': 0.1}


Вот теперь RMSE ненамного, но все же меньше 0,87, поэтому на этом этапе можно остановиться. Интересно, что набор параметров best_params получился таким же, как и для уменьшенного датасета, только с большим количеством эпох  